In [47]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain.globals import set_debug
from typing import List, Sequence
from langgraph.graph import MessageGraph, END



In [54]:
def create_agent(number):
    # This function is used to create an agent 
    llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature = 0.5)
    llm_with_tools = llm.bind_functions([SubQuery])
    prompt = ChatPromptTemplate.from_messages([("system", system_message), MessagesPlaceholder(variable_name = "messages")])
    request = HumanMessage(content =human)
    
    llm_response = ""
    if number == 1:
        parser = PydanticToolsParser(tools=[SubQuery])
        chain = prompt | llm_with_tools | parser
        for chunk in chain.stream({"messages":[request]}):
            print(chunk)
            output = chunk[0].sub_query
            print(output)    
  

    elif number == 2:
        parser = StrOutputParser()
        chain = prompt | llm | parser
        for chunk in chain.stream({"messages":[request]}):
            output = chunk
            print(output)
        return



#Definition of subquery
class SubQuery(BaseModel):
    '''Search for the geometric definition of a feature across the vector database'''

    sub_query: str = Field(
        ...,
        description="The text to be used as a sub-query in the prompt.",
    )   

#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "VertexLang"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    return "\n".join(lines)

#fewshot prompt definition
input_examples = context_gen("inputex.txt")
output_examples = context_gen("outputex.txt")
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

human = ("Select all vanes")


In [55]:
#first agent 
system_message = context_gen("system2.txt")
create_agent(1)


[]


IndexError: list index out of range

NameError: name 'output' is not defined